In [2]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn import linear_model
from sklearn.cluster import KMeans
from mpl_toolkits import mplot3d
import re

In [148]:
set_info = pd.read_csv("C:/Users/thoma/Desktop/Magic Project/data/AllPrintingsCSVFiles/sets.csv",usecols= ['name','code','releaseDate'])
set_info.columns = ['setCode','setName','releaseDate']
card_info = pd.read_csv("C:/Users/thoma/Desktop/Magic Project/data/AllPrintingsCSVFiles/cards.csv",usecols=["name","setCode","edhrecRank","convertedManaCost","isReserved","originalReleaseDate","rarity","text","colors"])
card_info.columns = ["colors","convertedManaCost","edhrecRank","isReserved","name","originalReleaseDate","rarity","setCode","text"]

C:\Users\thoma\AppData\Local\Temp\ipykernel_7732\4278852833.py:3: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  card_info = pd.read_csv("C:/Users/thoma/Desktop/Magic Project/data/AllPrintingsCSVFiles/cards.csv",usecols=["name","setCode","edhrecRank","convertedManaCost","isReserved","originalReleaseDate","rarity","text","colors"])


In [149]:
card_info

,colors,convertedManaCost,edhrecRank,isReserved,name,originalReleaseDate,rarity,setCode,text
0,W,7.0,15723.0,0,Ancestor's Chosen,NaN,uncommon,10E,First strike (This creature deals combat damag...
1,W,5.0,12809.0,0,Angel of Mercy,NaN,uncommon,10E,Flying\nWhen Angel of Mercy enters the battlef...
2,W,4.0,11227.0,0,Aven Cloudchaser,NaN,common,10E,Flying (This creature can't be blocked except ...
3,W,4.0,12901.0,0,Ballista Squad,NaN,uncommon,10E,"{X}{W}, {T}: Ballista Squad deals X damage to ..."
4,W,1.0,3975.0,0,Bandage,NaN,common,10E,Prevent the next 1 damage that would be dealt ...
...,...,...,...,...,...,...,...,...,...
56776,U,2.0,1147.0,0,Into the Roil,NaN,common,ZNR,Kicker {1}{U} (You may pay an additional {1}{U...
56777,B,1.0,8902.0,0,Bloodchief's Thirst,NaN,uncommon,ZNR,Kicker {2}{B} (You may pay an additional {2}{B...
56778,R,2.0,14041.0,0,Roil Eruption,NaN,common,ZNR,Kicker {5} (You may pay an additional {5} as y...
56779,G,3.0,1888.0,0,Roiling Regrowth,NaN,uncommon,ZNR,Sacrifice a land. Search your library for up t...


In [5]:
set_info

,setCode,setName,releaseDate
0,10E,Tenth Edition,2007-07-13
1,2ED,Unlimited Edition,1993-12-01
2,2XM,Double Masters,2020-08-07
3,3ED,Revised Edition,1994-04-01
4,4BB,Fourth Edition Foreign Black Border,1995-04-01
...,...,...,...
548,XLN,Ixalan,2017-09-29
549,ZEN,Zendikar,2009-10-02
550,ZNC,Zendikar Rising Commander,2020-09-25
551,ZNE,Zendikar Rising Expeditions,2020-09-25


In [66]:
set_info.loc[set_info['setCode'].isin(['UNH'])]

,setCode,setName,releaseDate
520,UNH,Unhinged,2004-11-19


In [68]:
card_info.loc[card_info['setCode'].isin(['UNH'])].sort_values(by = 'convertedManaCost')

,convertedManaCost,edhrecRank,isReserved,name,originalReleaseDate,rarity,setCode,text
52680,0.0,NaN,0,Island,NaN,common,UNH,({T}: Add {U}.)
52559,0.0,NaN,0,City of Ass,NaN,rare,UNH,City of Ass enters the battlefield tapped.\n{T...
52683,0.0,NaN,0,Forest,NaN,common,UNH,({T}: Add {G}.)
52682,0.0,NaN,0,Mountain,NaN,common,UNH,({T}: Add {R}.)
52681,0.0,NaN,0,Swamp,NaN,common,UNH,({T}: Add {B}.)
...,...,...,...,...,...,...,...,...
52664,13.0,NaN,0,Who // What // When // Where // Why,NaN,rare,UNH,Target player gains X life.
52625,15.0,NaN,0,Mox Lotus,NaN,rare,UNH,{T}: Add {∞}.\n{100}: Add one mana of any color.
52624,15.0,NaN,0,Mox Lotus,NaN,rare,UNH,{T}: Add {∞}.\n{100}: Add one mana of any color.
52587,1000000.0,NaN,0,Gleemax,NaN,rare,UNH,You choose all targets for all spells and abil...


In [105]:
card_info['edhrecRank'].max()

20898.0

In [110]:
values = {"convertedManaCost":0.0  ,"edhrecRank" : 20899.0, "isReserved" : 0, "name": 'loool', "originalReleaseDate": 7, "rarity": 'common',"setCode": 'Please Label set','text':'please enter text'}
card_info.fillna(value = values,inplace = True)

In [103]:
card_info.groupby(['setCode']).get_group('10E')['edhrecRank'].nsmallest(5).values

array([12., 23., 29., 47., 58.])

In [156]:
data_m = pd.DataFrame()

In [163]:
data_m

,B,"B,G","B,G,R","B,G,R,U,W","B,G,U","B,G,W","B,R","B,R,U","B,R,W","B,U",...,"G,U","G,U,W","G,W",R,"R,U","R,U,W","R,W",U,"U,W",W
0,31.0,155.0,1454.0,2137.0,699.0,1370.0,259.0,1196.0,1022.0,377.0,...,247.0,1566.0,225.0,43.0,514.0,1549.0,180.0,17.0,153.0,14.0
1,31.0,155.0,1454.0,2137.0,699.0,1370.0,259.0,1196.0,1022.0,377.0,...,247.0,1566.0,225.0,43.0,514.0,1549.0,180.0,17.0,153.0,14.0
2,31.0,155.0,1603.0,2137.0,699.0,1370.0,259.0,1196.0,1022.0,377.0,...,247.0,1566.0,225.0,43.0,514.0,1549.0,180.0,17.0,240.0,14.0
3,31.0,155.0,1603.0,2137.0,699.0,1370.0,259.0,1673.0,1022.0,392.0,...,247.0,1754.0,298.0,43.0,636.0,1684.0,180.0,17.0,240.0,14.0
4,31.0,203.0,1603.0,2317.0,859.0,1816.0,259.0,1673.0,1022.0,392.0,...,250.0,1754.0,298.0,43.0,636.0,1684.0,180.0,17.0,240.0,14.0
5,31.0,203.0,2731.0,2317.0,859.0,1816.0,259.0,1673.0,1214.0,392.0,...,250.0,2003.0,298.0,43.0,636.0,1725.0,180.0,17.0,240.0,14.0
6,31.0,203.0,2731.0,2317.0,859.0,1816.0,259.0,1673.0,1214.0,392.0,...,250.0,2003.0,298.0,43.0,636.0,1725.0,180.0,17.0,240.0,14.0
7,31.0,203.0,2731.0,2576.0,859.0,1816.0,259.0,1673.0,1214.0,392.0,...,352.0,2115.0,298.0,43.0,636.0,2042.0,180.0,17.0,459.0,14.0
8,31.0,203.0,2731.0,2748.0,1317.0,2220.0,259.0,1673.0,1214.0,392.0,...,352.0,2115.0,298.0,43.0,636.0,2529.0,657.0,17.0,459.0,14.0
9,31.0,203.0,2923.0,2748.0,1317.0,2220.0,259.0,1673.0,1755.0,392.0,...,352.0,2115.0,298.0,43.0,644.0,2529.0,657.0,17.0,459.0,14.0


In [121]:
card_info.groupby(['setCode']).get_group(group)['edhrecRank'].nsmallest(5).size

2

In [ ]:
data_m = pd.DataFrame()
n_small = 20
for group in card_info.groupby(['colors']).groups:
    if card_info.groupby(['colors']).get_group(group)['edhrecRank'].nsmallest(n_small).size >= n_small:
        data_m[group] = card_info.groupby(['colors']).get_group(group)['edhrecRank'].nsmallest(n_small).values
        print(data_m)

In [23]:
expanded_info = pd.merge(set_info,card_info.groupby(['setCode']).mean(), left_on = 'setCode', right_on = 'setCode')

In [32]:
expanded_info['convertedManaCost'].sort_values()

371        0.000000
369        0.000000
372        0.000000
490        0.000000
373        0.000000
           ...     
274        7.000000
88         7.400000
388        8.000000
254        9.000000
505    11564.234104
Name: convertedManaCost, Length: 538, dtype: float64

In [182]:
card_info.drop(card_info.index[[0,1]])

,colors,convertedManaCost,edhrecRank,isReserved,name,originalReleaseDate,rarity,setCode,text
2,W,4.0,11227.0,0,Aven Cloudchaser,NaN,common,10E,Flying (This creature can't be blocked except ...
3,W,4.0,12901.0,0,Ballista Squad,NaN,uncommon,10E,"{X}{W}, {T}: Ballista Squad deals X damage to ..."
4,W,1.0,3975.0,0,Bandage,NaN,common,10E,Prevent the next 1 damage that would be dealt ...
5,W,6.0,2809.0,0,Beacon of Immortality,NaN,rare,10E,Double target player's life total. Shuffle Bea...
6,W,3.0,13054.0,0,Benalish Knight,NaN,common,10E,Flash (You may cast this spell any time you co...
...,...,...,...,...,...,...,...,...,...
56776,U,2.0,1147.0,0,Into the Roil,NaN,common,ZNR,Kicker {1}{U} (You may pay an additional {1}{U...
56777,B,1.0,8902.0,0,Bloodchief's Thirst,NaN,uncommon,ZNR,Kicker {2}{B} (You may pay an additional {2}{B...
56778,R,2.0,14041.0,0,Roil Eruption,NaN,common,ZNR,Kicker {5} (You may pay an additional {5} as y...
56779,G,3.0,1888.0,0,Roiling Regrowth,NaN,uncommon,ZNR,Sacrifice a land. Search your library for up t...


In [193]:
card_info.index[card_info.iloc[[356, 357, 358, 359, 798, 799, 800, 1184, 1185]].index]

Int64Index([356, 357, 358, 359, 798, 799, 800, 1184, 1185], dtype='int64')

In [197]:
card_info.drop(card_info.index[[356, 357, 358, 359, 798, 799, 800, 1184, 1185]])

,colors,convertedManaCost,edhrecRank,isReserved,name,originalReleaseDate,rarity,setCode,text
0,W,7.0,15723.0,0,Ancestor's Chosen,NaN,uncommon,10E,First strike (This creature deals combat damag...
1,W,5.0,12809.0,0,Angel of Mercy,NaN,uncommon,10E,Flying\nWhen Angel of Mercy enters the battlef...
2,W,4.0,11227.0,0,Aven Cloudchaser,NaN,common,10E,Flying (This creature can't be blocked except ...
3,W,4.0,12901.0,0,Ballista Squad,NaN,uncommon,10E,"{X}{W}, {T}: Ballista Squad deals X damage to ..."
4,W,1.0,3975.0,0,Bandage,NaN,common,10E,Prevent the next 1 damage that would be dealt ...
...,...,...,...,...,...,...,...,...,...
56776,U,2.0,1147.0,0,Into the Roil,NaN,common,ZNR,Kicker {1}{U} (You may pay an additional {1}{U...
56777,B,1.0,8902.0,0,Bloodchief's Thirst,NaN,uncommon,ZNR,Kicker {2}{B} (You may pay an additional {2}{B...
56778,R,2.0,14041.0,0,Roil Eruption,NaN,common,ZNR,Kicker {5} (You may pay an additional {5} as y...
56779,G,3.0,1888.0,0,Roiling Regrowth,NaN,uncommon,ZNR,Sacrifice a land. Search your library for up t...


In [202]:
basics_removed = card_info.drop(card_info.loc[card_info['name'].isin(['Island','Mountain','Swamp','Plains','Forest'])].index)

In [203]:
basics_removed

,colors,convertedManaCost,edhrecRank,isReserved,name,originalReleaseDate,rarity,setCode,text
0,W,7.0,15723.0,0,Ancestor's Chosen,NaN,uncommon,10E,First strike (This creature deals combat damag...
1,W,5.0,12809.0,0,Angel of Mercy,NaN,uncommon,10E,Flying\nWhen Angel of Mercy enters the battlef...
2,W,4.0,11227.0,0,Aven Cloudchaser,NaN,common,10E,Flying (This creature can't be blocked except ...
3,W,4.0,12901.0,0,Ballista Squad,NaN,uncommon,10E,"{X}{W}, {T}: Ballista Squad deals X damage to ..."
4,W,1.0,3975.0,0,Bandage,NaN,common,10E,Prevent the next 1 damage that would be dealt ...
...,...,...,...,...,...,...,...,...,...
56776,U,2.0,1147.0,0,Into the Roil,NaN,common,ZNR,Kicker {1}{U} (You may pay an additional {1}{U...
56777,B,1.0,8902.0,0,Bloodchief's Thirst,NaN,uncommon,ZNR,Kicker {2}{B} (You may pay an additional {2}{B...
56778,R,2.0,14041.0,0,Roil Eruption,NaN,common,ZNR,Kicker {5} (You may pay an additional {5} as y...
56779,G,3.0,1888.0,0,Roiling Regrowth,NaN,uncommon,ZNR,Sacrifice a land. Search your library for up t...
